Syntax natürlicher Sprachen, WS 2023/24

# 11 - Aufgabenblatt (Lösung)

In [1]:
import nltk
from nltk import Tree

## Aufgabe 1 - Lexikalisierte CFG

#### Gegeben sei folgende FCFG (für einen Satz mit Kasusadverbial)  mit nur teilweise durchgeführter Kopfannotation über ein `HEAD`-Merkmal.

#### Vervollständigen Sie die Kopfannotation gemäß UD-Regeln.

In [2]:
sentence = "er rennt den ganzen Tag"

gramstring = r"""
% start S
    S[HEAD=?x] -> NP[] VP[HEAD=?x]
    NP[HEAD=?x] -> PRON[HEAD=?x]
    NP[HEAD=?x] -> DET[] ADJ[] N[HEAD=?x]
    VP[HEAD=?x] -> V[HEAD=?x]
    VP[HEAD=?x] -> VP[HEAD=?x] NP[]

    PRON[HEAD='er'] -> "er"
    DET[HEAD='den'] -> "den"
    ADJ[HEAD='ganz'] -> "ganzen"
    N[HEAD='Tag'] -> "Tag"
    V[HEAD='rennt'] -> "rennt"
"""

grammar = nltk.grammar.FeatureGrammar.fromstring(gramstring)
parser = nltk.parse.FeatureChartParser(grammar,trace=0)

for tree in parser.parse(sentence.split()):
    print(tree)
    tree = Tree.fromstring(str(tree).replace(", ",","))
    tree.pretty_print(unicodelines=True)
    #display(tree)

(S[HEAD='rennt']
  (NP[HEAD='er'] (PRON[HEAD='er'] er))
  (VP[HEAD='rennt']
    (VP[HEAD='rennt'] (V[HEAD='rennt'] rennt))
    (NP[HEAD='Tag']
      (DET[HEAD='den'] den)
      (ADJ[HEAD='ganz'] ganzen)
      (N[HEAD='Tag'] Tag))))
                                 S[HEAD='rennt']                                                
       ┌────────────────────────────────┴───────────────┐                                        
       │                                         VP[HEAD='rennt']                               
       │               ┌────────────────────────────────┴────────────────┐                       
 NP[HEAD='er']  VP[HEAD='rennt']                                   NP[HEAD='Tag']               
       │               │                ┌────────────────────────────────┼───────────────┐       
PRON[HEAD='er'] V[HEAD='rennt']  DET[HEAD='den']                  ADJ[HEAD='ganz'] N[HEAD='Tag']
       │               │                │                                │            

## Aufgabe 2 - Parent Annotation

#### Gegeben sei folgende CFG (für einen ditransitiven Satz) mit unvollständiger *Parent Annotation*.

#### Führen Sie über Symbolerweiterung (mit `^` als Trennerzeichen) eine *Parent Annotation* durch, wie sie durch die Regelanwendungen im Syntaxbaum der Angabe impliziert ist.

##### Sie können sich in der Annotation auf die syntaktischen Regeln beschränken.


In [3]:
sentence = "Kim gave him a bone"

grammar = nltk.CFG.fromstring("""
    S   -> NP^S VP^S
    VP^S  -> V NP^VP NP^VP
    NP^S  -> PROPN
    NP^VP  -> PRON
    NP^VP  -> DET N

    DET -> "a"
    PROPN  -> "Kim"
    PRON   -> "him"
    N      -> "bone"
    V      -> "gave"
""")

parser = nltk.ChartParser(grammar,trace=0)

for tree in parser.parse(sentence.split()):
    tree.pretty_print(unicodelines=True)

       S                       
  ┌────┴─────┐                  
  │         VP^S               
  │    ┌─────┼─────────┐        
 NP^S  │   NP^VP     NP^VP     
  │    │     │    ┌────┴────┐   
PROPN  V    PRON DET        N  
  │    │     │    │         │   
 Kim  gave  him   a        bone



## Aufgabe 3 - Disambiguierung mit PCFG

### a) Gegeben sei folgende Mini-Treebank mit verschiedenen Subkategorisierungtypen (Kasusadverbial vs. Objekt):

In [5]:
treestrings = [
"""(S
  (NP (PRON er))
  (VP
    (VP (V rennt))
    (NP
      (DET den)
      (ADJ ganzen)
      (N Tag))))
""",

"""(S
  (NP (PRON er))
  (VP
    (V verschwendet)
    (NP
      (DET den)
      (ADJ ganzen)
      (N Tag))))
""",

]

trees = [Tree.fromstring(treestring).pretty_print(unicodelines=True) for treestring in treestrings]

            S                
 ┌──────────┴───┐             
 │              VP           
 │     ┌────────┴────┐        
 NP    VP            NP      
 │     │    ┌────────┼─────┐  
PRON   V   DET      ADJ    N 
 │     │    │        │     │  
 er  rennt den     ganzen Tag

                   S                
 ┌─────────────────┴───┐             
 │                     VP           
 │        ┌────────────┴────┐        
 NP       │                 NP      
 │        │        ┌────────┼─────┐  
PRON      V       DET      ADJ    N 
 │        │        │        │     │  
 er  verschwendet den     ganzen Tag



#### Passen Sie diese Mini-Treebank durch Duplizieren einer minimalen Anzahl an Sätzen so an, dass die daraus induzierte Grammatik für den ersten Ableitungstyp eine korrekte Analyse liefert.

In [6]:
treestrings = [
"""(S
  (NP (PRON er))
  (VP
    (VP (V rennt))
    (NP
      (DET den)
      (ADJ ganzen)
      (N Tag))))
""",

"""(S
  (NP (PRON er))
  (VP
    (VP (V rennt))
    (NP
      (DET den)
      (ADJ ganzen)
      (N Tag))))
""", #dupliziert

"""(S
  (NP (PRON er))
  (VP
    (VP (V rennt))
    (NP
      (DET den)
      (ADJ ganzen)
      (N Tag))))
""", #dupliziert




"""(S
  (NP (PRON er))
  (VP
    (V verschwendet)
    (NP
      (DET den)
      (ADJ ganzen)
      (N Tag))))
""",

]


trees = [Tree.fromstring(treestring) for treestring in treestrings]

#grammar induction:    
productions = []
S = nltk.Nonterminal('S')

for tree in trees:
    productions += tree.productions()

grammar = nltk.induce_pcfg(S, productions)
for production in grammar.productions():
    print(production)    


#parse first tree with grammar:
parser = nltk.ViterbiParser(grammar)
for parse in parser.parse(trees[0].leaves()):
    print(parse)
    parse.pretty_print(unicodelines=True)

S -> NP VP [1.0]
NP -> PRON [0.5]
PRON -> 'er' [1.0]
VP -> VP NP [0.428571]
VP -> V [0.428571]
V -> 'rennt' [0.75]
NP -> DET ADJ N [0.5]
DET -> 'den' [1.0]
ADJ -> 'ganzen' [1.0]
N -> 'Tag' [1.0]
VP -> V NP [0.142857]
V -> 'verschwendet' [0.25]
(S
  (NP (PRON er))
  (VP (VP (V rennt)) (NP (DET den) (ADJ ganzen) (N Tag)))) (p=0.0344388)
            S                
 ┌──────────┴───┐             
 │              VP           
 │     ┌────────┴────┐        
 NP    VP            NP      
 │     │    ┌────────┼─────┐  
PRON   V   DET      ADJ    N 
 │     │    │        │     │  
 er  rennt den     ganzen Tag



#### Parsen Sie abschließend die Sätze mit der induzierten Grammatik:

In [7]:
#parse trees with grammar:
parser = nltk.ViterbiParser(grammar)
for tree in trees:
    for parse in parser.parse(tree.leaves()):
        #print(parse)
        parse.pretty_print(unicodelines=True)

            S                
 ┌──────────┴───┐             
 │              VP           
 │     ┌────────┴────┐        
 NP    VP            NP      
 │     │    ┌────────┼─────┐  
PRON   V   DET      ADJ    N 
 │     │    │        │     │  
 er  rennt den     ganzen Tag

            S                
 ┌──────────┴───┐             
 │              VP           
 │     ┌────────┴────┐        
 NP    VP            NP      
 │     │    ┌────────┼─────┐  
PRON   V   DET      ADJ    N 
 │     │    │        │     │  
 er  rennt den     ganzen Tag

            S                
 ┌──────────┴───┐             
 │              VP           
 │     ┌────────┴────┐        
 NP    VP            NP      
 │     │    ┌────────┼─────┐  
PRON   V   DET      ADJ    N 
 │     │    │        │     │  
 er  rennt den     ganzen Tag

                   S                
 ┌─────────────────┴───┐             
 │                     VP           
 │        ┌────────────┴────┐        
 NP       VP             

### b) Verbesserung durch Lexikalisierung

#### Führen Sie in der ursprünglichen Mini-Treebank eine teilweise Lexikalisierung durch Kopfannotation durch, so dass je nach Verb die entsprechende Konstruktion (`VP -> VP NP` vs. `VP -> V NP`) bevorzugt wird.

In [10]:
treestrings = [
"""(S
  (NP (PRON er))
  (VP
    (VP (V_rennt rennt))
    (NP
      (DET den)
      (ADJ ganzen)
      (N Tag))))
""",

"""(S
  (NP (PRON er))
  (VP
    (V_verschwendet verschwendet)
    (NP
      (DET den)
      (ADJ ganzen)
      (N Tag))))
""",

]


trees = [Tree.fromstring(treestring) for treestring in treestrings]

#grammar induction:
productions = []
S = nltk.Nonterminal('S')

for tree in trees:
    productions += tree.productions()

grammar = nltk.induce_pcfg(S, productions)
for production in grammar.productions():
    print(production)    

#parse trees with grammar:
parser = nltk.ViterbiParser(grammar)
for tree in trees:
    for parse in parser.parse(tree.leaves()):
        print(parse)
        parse.pretty_print(unicodelines=True)

S -> NP VP [1.0]
NP -> PRON [0.5]
PRON -> 'er' [1.0]
VP -> VP NP [0.333333]
VP -> V_rennt [0.333333]
V_rennt -> 'rennt' [1.0]
NP -> DET ADJ N [0.5]
DET -> 'den' [1.0]
ADJ -> 'ganzen' [1.0]
N -> 'Tag' [1.0]
VP -> V_verschwendet NP [0.333333]
V_verschwendet -> 'verschwendet' [1.0]
(S
  (NP (PRON er))
  (VP
    (VP (V_rennt rennt))
    (NP (DET den) (ADJ ganzen) (N Tag)))) (p=0.0277778)
              S                
 ┌────────────┴───┐             
 │                VP           
 │      ┌─────────┴────┐        
 NP     VP             NP      
 │      │     ┌────────┼─────┐  
PRON V_rennt DET      ADJ    N 
 │      │     │        │     │  
 er   rennt  den     ganzen Tag

(S
  (NP (PRON er))
  (VP
    (V_verschwendet verschwendet)
    (NP (DET den) (ADJ ganzen) (N Tag)))) (p=0.0833333)
                     S                
 ┌───────────────────┴───┐             
 │                       VP           
 │         ┌─────────────┴────┐        
 NP        │                  NP      
 │     